In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import datetime as dt


import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

In [2]:
train = pd.read_csv('../01_Data/train.csv', parse_dates = ['booking_date','checkin_date','checkout_date'])
test = pd.read_csv('../01_Data/test.csv', parse_dates = ['booking_date','checkin_date','checkout_date'])

In [3]:
def date_cleaning(df):
    format_str = '%d/%m/%Y' # The format
    df['booking_date'] = df['booking_date'].apply(lambda x: dt.datetime.strftime(x, format_str))
    df['checkin_date'] = df['checkin_date'].apply(lambda x: dt.datetime.strftime(x, format_str))
    df['checkout_date'] = df['checkout_date'].apply(lambda x: dt.datetime.strftime(x, format_str))
    df['booking_date'] = pd.to_datetime(df['booking_date'])
    df['checkin_date'] = pd.to_datetime(df['checkin_date'])
    df['checkout_date'] = pd.to_datetime(df['checkout_date'])    
    df['D_checkin_month'] =  df.checkin_date.dt.month
    df['D_checkin_year'] =  df.checkin_date.dt.year
    df['D_checkin_week'] =  df.checkin_date.dt.week
    df['D_checkin_weekday'] =  df.checkin_date.dt.weekday
    df['D_booking_month'] =  df.booking_date.dt.month
    df['D_booking_year'] =  df.booking_date.dt.year
    df['D_booking_week'] =  df.booking_date.dt.week
    df['D_booking_weekday'] =  df.booking_date.dt.weekday
    df['checkin_date'] = np.where(df.D_checkin_year < 2015, df['checkin_date'] + pd.DateOffset(years=6), df['checkin_date'])
    df['checkout_date'] = np.where(df.D_checkin_year < 2015, df['checkout_date'] + pd.DateOffset(years=6), df['checkout_date'])
    df['D_checkin_year'] =  df.checkin_date.dt.year
    df['D_no_days_prior_Advanced'] =  (df.checkin_date - df.booking_date).dt.days
    df['D_no_days_stay'] =  (df.checkout_date - df.checkin_date).dt.days
    
    return df

In [4]:
train = date_cleaning(train)
test = date_cleaning(test)

In [5]:
member_id_info = pd.DataFrame()
member_id_info['memberid'] = train.memberid.unique()
member_id_info['tag'] = np.random.rand(member_id_info.shape[0])
member_id_info['val_tag'] = np.random.randint(1,6,member_id_info.shape[0])
member_id_info['tag'] = np.where(member_id_info.tag>0.2,"Train","Val")
member_id_info.head()

,memberid,tag,val_tag
0,3d1539e56495b6991f0a3ef5a61ca3d03ce4fff7380e9a...,Val,5
1,7b68caeb153ac83aeab0b9a679709c558d57a6c9d2bdf6...,Train,3
2,0fb5e9e87638ac401e88ea0115114182614151218a0147...,Train,4
3,383d253fc6056a0da3fdc32703b4816869f6a3bf8d14ae...,Train,3
4,d4eee99d5c3238cdda52479fb34125e15208d394af34e7...,Train,2


In [6]:
train = pd.merge(train, member_id_info, on= ['memberid'], how = "left")
train[train.tag == "Train"].val_tag.value_counts(normalize = True)

1    0.201430
2    0.201203
4    0.200176
5    0.199891
3    0.197300
Name: val_tag, dtype: float64

In [7]:
test['tag'] = ['Test']*test.shape[0]
test['val_tag'] = [0]*test.shape[0]
test['amount_spent_per_room_night_scaled'] = [0]*test.shape[0]

In [8]:
train_test = pd.concat([train,test], axis=0)

In [9]:
def booking_info(df):
    df['checkin_year_week'] = df['D_checkin_year'].map(str) + df['D_checkin_week'].map(str)
    df['booking_year_week'] = df['D_booking_year'].map(str) + df['D_booking_week'].map(str)
    
    
    multiple_res_same_day = df.groupby(['memberid','checkin_date'])['amount_spent_per_room_night_scaled'].count().reset_index()
    multiple_res_same_day.rename(columns= {"amount_spent_per_room_night_scaled":"D_res_cnt_same_day"}, inplace = True)
    df = pd.merge(df,multiple_res_same_day, on = ['memberid','checkin_date'], how = "left")
    
    multiple_book_same_day = df.groupby(['memberid','booking_date'])['amount_spent_per_room_night_scaled'].count().reset_index()
    multiple_book_same_day.rename(columns= {"amount_spent_per_room_night_scaled":"D_book_cnt_same_day"}, inplace = True)
    df = pd.merge(df,multiple_book_same_day, on = ['memberid','booking_date'], how = "left")
    
    multiple_res_same_week = df.groupby(['memberid','checkin_year_week'])['amount_spent_per_room_night_scaled'].count().reset_index()
    multiple_res_same_week.rename(columns= {"amount_spent_per_room_night_scaled":"D_res_cnt_same_week"}, inplace = True)
    df = pd.merge(df,multiple_res_same_week, on = ['memberid','checkin_year_week'], how = "left")
    
    multiple_book_same_week = df.groupby(['memberid','booking_year_week'])['amount_spent_per_room_night_scaled'].count().reset_index()
    multiple_book_same_week.rename(columns= {"amount_spent_per_room_night_scaled":"D_book_cnt_same_week"}, inplace = True)
    df = pd.merge(df,multiple_book_same_week, on = ['memberid','booking_year_week'], how = "left")
    
    multiple_res = df.groupby(['memberid'])['amount_spent_per_room_night_scaled'].count().reset_index()
    multiple_res.rename(columns= {"amount_spent_per_room_night_scaled":"D_res_cnt"}, inplace = True)
    df = pd.merge(df,multiple_res, on = ['memberid'], how = "left")
    
    df.drop(['checkin_year_week','booking_year_week'], axis=1, inplace = True)
    return df
     

In [10]:
train_test = booking_info(train_test)

In [11]:
train_test.to_csv('../03_Feature_Engineering/v10.csv', index = False)